In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
import time
import re
import os
import requests

In [ ]:
def scarpe(path):
    options = ChromeOptions()
    options.headless=True
    service = Service(ChromeDriverManager().install())
    driver = Chrome(service=service, options=options)
    driver.get(path)
    # driver.maximize_window()
    
    # number of cars
    try:
        # time.sleep(5)
        cars_tag = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/p[@class="bodycopy"]')))
        if cars_tag:
            strongs = cars_tag.find_elements(By.TAG_NAME, "strong")
            # get the last strong value
            cars = strongs[-1].text.strip()
            # print(cars)
            splited_cars = cars.split(": ")
            total_cars = int(splited_cars[1])
            print(f"{total_cars} number of cars found")
        else:
            print("No cars found")
    except Exception as e:
        print(f"No cars found and")
    
    
    # result
    results = []
    car_count = 0
    visited_pages = {"1"} # the first page already visited
    while car_count < 52: #total_cars
        
        try:
            page_cars = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/a[text() = "More Info"]')))
            
            for i in range(len(page_cars)): # len(page_cars)
                if car_count >= total_cars:
                    break
                
                try:
                    page_cars = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/a[text() = "More Info"]')))
                    driver.execute_script("arguments[0].scrollIntoView();", page_cars[i])
                    time.sleep(1)
                    page_cars[i].click()

                    details = {}    
                    
                    # car details complete
                    try:
                        det_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="stockdetailwrap"]')))
                        if det_card:
                            
                            # lot number--> done
                            try:
                                lot_trs = WebDriverWait(det_card, 5).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tr'))) 
                                for lot_tr in lot_trs:
                                    ths = lot_tr.find_elements(By.TAG_NAME, 'th')
                                    if len(ths) >=2:
                                        lot = ths[1].text.strip()
                                        if ths:
                                            details['lot'] = lot.split(" ")[2]
                                        else:
                                            details['lot'] = "na"
                            except Exception as e:
                                print("No lot number")
                            
                            # other information --> done
                            try:
                                det_trs =WebDriverWait(det_card, 5).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tr'))) 
                                if det_trs:
                                    for det_tr in det_trs:
                                        det_tds = det_tr.find_elements(By.TAG_NAME, 'td')
                                        if len(det_tds) == 2:
                                            details[det_tds[0].text.strip()] = det_tds[1].text.strip()
                                else:
                                    print("No details")
                                    
                                # images
                                try:
                                    img_as = WebDriverWait(det_card, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/a[@style="border-width: 2px;"]')))
                                    if img_as:
                                        imgs_lst = []
                                        for img_a in img_as:
                                            imgs = img_a.find_element(By.TAG_NAME, "img")
                                            imgs_lst.append(imgs.get_attribute("src"))
                                            details['Images'] = ", ".join(imgs_lst)
                                    else:
                                        details['Images'] = "na"
                                except Exception as e:
                                    print("No images found")
                                         
                            except Exception as e:
                                print(f"No details")   
                                                         
                    except Exception as e:
                        print("No details main card card found") 
                               
                    car_count+=1
                    results.append(details)
                    driver.back()
                
                except Exception as e:
                    print(f"No more cars found {e}")
                    
            
            if car_count % 10 == 0: # --> 25%25 == 0 and 50%25== 0 --> issue here
                try:
                    next_card =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//tr[@class="stockPagination pagination"])[2]'))) 
                    next_link = WebDriverWait(next_card, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/a[not(contains(@class,"selected"))]')))
                    if next_link:
                        for nxt_btn in next_link:
                            page_text = nxt_btn.text.strip()
                            if page_text and page_text not in visited_pages:  # Check if the page has not been visited
                                # print(f"Visiting: {page_url}")
                                visited_pages.add(page_text)  # Mark it as visited
                                print(f"Visiting page {page_text}")
                                nxt_btn.click()
                                
                                time.sleep(2)
                                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/a[text() = "More Info"]')))
                                print("Hit the next button")
                                break
                            
                except Exception as e:
                    print("No next found")
                    break 
        except Exception as e:
            print("No page cars found")
            break
            
    
    df = pd.DataFrame.from_dict(results)
    df.to_csv("smva.csv", index=False)
    
    # time.sleep(2)
    driver.quit()
path = "https://www.smva.co.uk/stock?sale=2281"
scarpe(path)

202 number of cars found
No images found
Visiting page 2
Hit the next button
No images found
No images found
No images found
Visiting page 3
Hit the next button
No images found
Visiting page 4
Hit the next button
No images found
No images found
No images found
No images found
Visiting page 5
Hit the next button
No images found
Visiting page 6
Hit the next button
No images found
Visiting page 7
Hit the next button


In [2]:
df = pd.read_csv("smva.csv")
df.head()

,lot,Vehicle,Registered,V5,Mileage,Description,MOT / Location,Road Tax Expiry / Location,Guide Value,VAT Status,No. of former Keepers,Additional Info,Images
0,1,FORD RANGER WILDTRAK ECOBLUE 4 209BHP - 1996cc,11 November 2019,Here,53889,Grey Automatic Diesel 4x4 Double Cab Pick Up,Nov 2025,-,"£13,500",Commercial Plus VAT,1.0,"5 STAMPS TO 45K, BOOKS AND SPARE KEYS IN THE O...",https://www.smva.co.uk/i_folder/i_folder/EJ69C...
1,2,MERCEDES-BENZ V220 SPORT BLUETEC AUTO 161BHP -...,28 June 2018,Here,120471,Silver Automatic Diesel MPV,Oct 2025,-,"£22,500",No VAT,2.0,SERVICE HISTORY TO 98K (3 INVOICES + PRINTOUT)...,https://www.smva.co.uk/i_folder/i_folder/LK18N...
2,30,HYUNDAI SANTA FE PREMIUM CRDI 194BHP - 2199cc,27 March 2012,Here,78933,Silver Manual Diesel Estate,Jan 2026,-,"£2,700",No VAT,1.0,"BOOKS, SPARE KEY",NaN
3,32,BMW 320D EFFICIENTDYNAMICS 161BHP - 1995cc,16 January 2014,T/A,138936,Blue Manual Diesel Estate,Feb 2026,-,"£3,100",No VAT,NaN,**CHERISHED TRANSFER** SERVICE HISTORY PRINTOU...,https://www.smva.co.uk/i_folder/i_folder/PF63E...
4,38,MAZDA MX-5 I ROADSTER VENTURE E 158BHP - 1999cc,13 May 2013,Here,83950,Red Manual Petrol Convertible,May 2025,-,"£4,950",No VAT,1.0,"SERVICE PRINTOUT TO 72K,BOOKS & SPARE KEY",https://www.smva.co.uk/i_folder/i_folder/CK13F...


### image downloader

In [6]:
from urllib.parse import urlparse, urljoin
# urlparse: Parses a URL into components (scheme, netloc, path, etc.), making it easy to validate or extract information from the URL.
# urljoin: Joins a base URL with a relative path to form a complete URL.

reg_img = df[["Vehicle", "Images"]]

def download_images(data, main_folder="Images"): 
    os.makedirs(main_folder, exist_ok=True)
    
    for index, row in data.iterrows():
        reg_no = row["Vehicle"]
        image_urls = row["Images"]

        # Check if 'Images' column is empty or NaN
        if pd.isna(image_urls) or not isinstance(image_urls, str) or image_urls.strip() == "":
            print(f"Skipping {reg_no} (No image URLs)")
            continue

        image_urls = image_urls.split(", ")  # Split URLs by comma
        
        reg_folder = os.path.join(main_folder, reg_no)
        os.makedirs(reg_folder, exist_ok=True)
        
        for idx, url in enumerate(image_urls):
            url = url.strip()
            if not url.startswith(("http://", "https://")):
                url = urljoin("https://", url) 
            
            parsed_url = urlparse(url)
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}") 
                continue
            
            try:
                response = requests.get(url, stream=True) 
                response.raise_for_status()
                
                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"
                file_extension = file_name.split(".")[-1]
                
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg"
                
                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")

# Call the function
download_images(reg_img)

Downloaded: Images\FORD RANGER WILDTRAK ECOBLUE 4 209BHP - 1996cc\FORD RANGER WILDTRAK ECOBLUE 4 209BHP - 1996cc_1_EJ69CZV.jpg
Downloaded: Images\FORD RANGER WILDTRAK ECOBLUE 4 209BHP - 1996cc\FORD RANGER WILDTRAK ECOBLUE 4 209BHP - 1996cc_2_EJ69CZV_1.jpg
Downloaded: Images\FORD RANGER WILDTRAK ECOBLUE 4 209BHP - 1996cc\FORD RANGER WILDTRAK ECOBLUE 4 209BHP - 1996cc_3_EJ69CZV_2.jpg
Downloaded: Images\FORD RANGER WILDTRAK ECOBLUE 4 209BHP - 1996cc\FORD RANGER WILDTRAK ECOBLUE 4 209BHP - 1996cc_4_EJ69CZV_3.jpg
Downloaded: Images\FORD RANGER WILDTRAK ECOBLUE 4 209BHP - 1996cc\FORD RANGER WILDTRAK ECOBLUE 4 209BHP - 1996cc_5_EJ69CZV_4.jpg
Downloaded: Images\FORD RANGER WILDTRAK ECOBLUE 4 209BHP - 1996cc\FORD RANGER WILDTRAK ECOBLUE 4 209BHP - 1996cc_6_EJ69CZV_5.jpg
Downloaded: Images\MERCEDES-BENZ V220 SPORT BLUETEC AUTO 161BHP - 2143cc\MERCEDES-BENZ V220 SPORT BLUETEC AUTO 161BHP - 2143cc_1_LK18NDL.jpg
Downloaded: Images\MERCEDES-BENZ V220 SPORT BLUETEC AUTO 161BHP - 2143cc\MERCEDES-BENZ 